In [ ]:
# define functions to generate plotly traces
from ROOT import TChain

def trajpoints_edepsim(filename,entry,trajectory):

    from ROOT import TChain
    ch=TChain("EDepSimEvents")
    ch.AddFile(filename)
    ch.GetEntry(entry)
    t=ch.Event.Trajectories.at(trajectory)
    # Let me convert points into numpy format cuz it's easier for me
    pts=[[pt.GetPosition().X()/10.,pt.GetPosition().Y()/10.,pt.GetPosition().Z()/10.,pt.GetPosition().T(),pt.GetMomentum().Mag()] for pt in t.Points]
    pts=np.array(pts).astype(np.float32)
    return pts

def energy_edepsim(filename, entry, trackid):
    from ROOT import TChain
    ch=TChain("EDepSimEvents")
    ch.AddFile(filename)
    ch.GetEntry(entry)

    lar_hits = ch.Event.SegmentDetectors["LArTracker"]
    pts = [[(h.GetStart().X()/10., h.GetStart().Y()/10., h.GetStart().Z()/10., h.GetEnergyDeposit()),
             (h.GetStop().X()/10., h.GetStop().Y()/10., h.GetStop().Z()/10., h.GetEnergyDeposit())]
             for h in lar_hits if h.GetPrimaryId() == trackid]
    
    pts=np.array(pts).astype(np.float32)
    return pts

def points_larcv(filename,entry,trajectory):
    ch=TChain("cluster3d_geant4_tree")
    ch.AddFile(filename)
    ch.GetEntries()

    # Access the same entry, same trajectory
    ch.GetEntry(entry)
    br=ch.cluster3d_geant4_branch
    vs=br.as_vector()[trajectory]
    meta=br.meta()

    # Convert larcv::VoxelSet into matrix coordinate
    x   = np.empty(shape=(vs.size(),), dtype=np.int32)
    y   = np.empty(shape=(vs.size(),), dtype=np.int32)
    z   = np.empty(shape=(vs.size(),), dtype=np.int32)
    val = np.empty(shape=(vs.size(),), dtype=np.float32)
    from larcv import larcv
    larcv.as_flat_arrays(vs,meta,x, y, z, val)
    # Convert to floating point positions
    pts = np.column_stack([x,y,z]).astype(np.float32)
    pts[:,0] = pts[:,0] * br.meta().size_voxel_x() + br.meta().min_x()
    pts[:,1] = pts[:,1] * br.meta().size_voxel_y() + br.meta().min_y()
    pts[:,2] = pts[:,2] * br.meta().size_voxel_z() + br.meta().min_z()
    pts = np.column_stack([pts,val.reshape((len(val),1))])
    return pts

# This is taken from lartpc_mlreco3d, so that lartpc_mlreco3d isn't required to make this plot
def scatter_cubes(coords, cubesize=1, color='orange', opacity=0.8, hovertext=None, colorscale=None, **kwargs):
    """
    Produces go.Mesh3d object to be plotted in plotly
    - coords is a list of cubes coordinates (Nx3 matrix)
    """
    base_x = np.array([0, 0, 1, 1, 0, 0, 1, 1]) * cubesize
    base_y = np.array([0, 1, 1, 0, 0, 1, 1, 0]) * cubesize
    base_z = np.array([0, 0, 0, 0, 1, 1, 1, 1]) * cubesize
    trace = []
    cmin, cmax = None, None
    if not isinstance(color, str):
        cmin = min(color)
        cmax = max(color)
    for i in range(len(coords)):
        trace.append(
                go.Mesh3d(
                    x=coords[i][0] + base_x,
                    y=coords[i][1] + base_y,
                    z=coords[i][2] + base_z,
                    i=[7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                    j=[3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                    k=[0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                    opacity=opacity,
                    color=color if isinstance(color, str) else color[i],
                    colorscale=colorscale,
                    cmin=cmin,
                    cmax=cmax,
                    hoverinfo=['x','y','z'] if hovertext is None else ['x', 'y', 'z','text'],
                    hovertext=hovertext,
                    **kwargs
                    )
                )
    return trace


In [ ]:
# import plotly + define base layout
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=False)

import plotly.graph_objs as go
layout = go.Layout(
showlegend=True,
legend=dict(x=1.01,y=0.95),
width=768,
height=768,
hovermode='closest',
margin=dict(l=0,r=0,b=0,t=0),                                                                                                                                  
#template='plotly_dark',                                                                                                                                        
uirevision = 'same',
scene = dict(xaxis = dict(nticks=10, range = (0,768), showticklabels=True, title='x'),
             yaxis = dict(nticks=10, range = (0,768), showticklabels=True, title='y'),
             zaxis = dict(nticks=10, range = (0,768), showticklabels=True, title='z'),
             aspectmode='cube')
)


In [ ]:
# Plot
traces = []

# # larcv
# pts = points_larcv('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/app/larcv2/larcv/app/Supera/supera.root',0,5)
# print(pts)
# # True to visualize voxels, or False to visualize voxel center points
# if True:
#     traces+=scatter_cubes(pts,cubesize=0.4434,color=pts[:,3],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
# else:
#     traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
#                                name='larcv',
#                                mode='markers',
#                                marker=dict(size=4, color=val, opacity=0.4),
#                                hovertext = ['E=%.2f MeV' % pts[i,3] for i in range(len(pts))]
#                               )
#                  )


#ede-sim
pts = trajpoints_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out.root',2,0)
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true traj',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkblue',width=2),
                           hovertext = ['P=%.2f MeV/c' % pts[i,4] for i in range(len(pts))]
                          )
                 )
traces.append(px.line_3d(pts)
              go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true traj',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkblue',width=2),
                           hovertext = ['P=%.2f MeV/c' % pts[i,4] for i in range(len(pts))]
                          )
                 )
pts = energy_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out.root', 2, 0)
diffs = [np.linalg.norm(np.array(pts[i][:3]) - np.array(pts[i+1][:3])) for i in range(len(pts)-1)]
counts, bins = np.histogram(diffs, bins=[0.1*i for i in range(50)])
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

#traces += scatter_cubes(pts,cubesize=0.4434,color=pts[:,4],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true edep',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkred',width=2),
                           hovertext = ['Edep=%.2f MeV' % pts[i,4] for i in range(len(pts))]
                          ))

# Shift the coordinates to show just a short track region
layout.scene.xaxis.range=(220,448)
layout.scene.yaxis.range=(-190,30)
layout.scene.zaxis.range=(650,650+220)


fig = go.Figure(data=traces)#,layout=layout)
fig.show()


In [ ]:
traces = []
pts = energy_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out-default.root', 2, 0)

diffs = [np.linalg.norm(np.array(pts[i][:3]) - np.array(pts[i+1][:3])) for i in range(len(pts)-1)]

counts, bins = np.histogram(diffs, bins=[0.1*i for i in range(50)])
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

#traces += scatter_cubes(pts,cubesize=0.4434,color=pts[:,4],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true edep',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkred',width=2),
                           hovertext = ['Edep=%.2f MeV' % pts[i,4] for i in range(len(pts))]
                          ))

# Shift the coordinates to show just a short track region
layout.scene.xaxis.range=(220,448)
layout.scene.yaxis.range=(-190,30)
layout.scene.zaxis.range=(650,650+220)


fig = go.Figure(data=traces)#,layout=layout)
fig.show()


In [ ]:
traces = []
pts = energy_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out-default.root', 2, 0)

diffs = [np.linalg.norm(np.array(pts[i][:3]) - np.array(pts[i+1][:3])) for i in range(len(pts)-1)]

counts, bins = np.histogram(diffs, bins=[0.1*i for i in range(50)])
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

#traces += scatter_cubes(pts,cubesize=0.4434,color=pts[:,4],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true edep',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkred',width=2),
                           hovertext = ['Edep=%.2f MeV' % pts[i,4] for i in range(len(pts))]
                          ))

# Shift the coordinates to show just a short track region
layout.scene.xaxis.range=(220,448)
layout.scene.yaxis.range=(-190,30)
layout.scene.zaxis.range=(650,650+220)


fig = go.Figure(data=traces)#,layout=layout)
fig.show()


In [ ]:
# Plot
traces = []

# # larcv
# pts = points_larcv('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/app/larcv2/larcv/app/Supera/supera.root',0,5)
# print(pts)
# # True to visualize voxels, or False to visualize voxel center points
# if True:
#     traces+=scatter_cubes(pts,cubesize=0.4434,color=pts[:,3],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
# else:
#     traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
#                                name='larcv',
#                                mode='markers',
#                                marker=dict(size=4, color=val, opacity=0.4),
#                                hovertext = ['E=%.2f MeV' % pts[i,3] for i in range(len(pts))]
#                               )
#                  )


#ede-sim
pts = trajpoints_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out.root',2,0)
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true traj',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkblue',width=2),
                           hovertext = ['P=%.2f MeV/c' % pts[i,4] for i in range(len(pts))]
                          )
                 )
traces.append(px.line_3d(pts)
              go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true traj',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkblue',width=2),
                           hovertext = ['P=%.2f MeV/c' % pts[i,4] for i in range(len(pts))]
                          )
                 )
pts = energy_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out.root', 2, 0)
diffs = [np.linalg.norm(np.array(pts[i][:3]) - np.array(pts[i+1][:3])) for i in range(len(pts)-1)]
counts, bins = np.histogram(diffs, bins=[0.1*i for i in range(50)])
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

#traces += scatter_cubes(pts,cubesize=0.4434,color=pts[:,4],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true edep',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkred',width=2),
                           hovertext = ['Edep=%.2f MeV' % pts[i,4] for i in range(len(pts))]
                          ))

# Shift the coordinates to show just a short track region
layout.scene.xaxis.range=(220,448)
layout.scene.yaxis.range=(-190,30)
layout.scene.zaxis.range=(650,650+220)


fig = go.Figure(data=traces)#,layout=layout)
fig.show()


In [ ]:
pts = energy_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out.root',2,0)
print(pts)
import pandas
df = pandas.df(pts)

In [ ]:
traces = []
pts = energy_edepsim('/gpfs/slac/staas/fs1/g/neutrino/jwolcott/data/scratch/out-default.root', 2, 0)

diffs = [np.linalg.norm(np.array(pts[i][:3]) - np.array(pts[i+1][:3])) for i in range(len(pts)-1)]

counts, bins = np.histogram(diffs, bins=[0.1*i for i in range(50)])
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

#traces += scatter_cubes(pts,cubesize=0.4434,color=pts[:,4],opacity=0.4,hovertext=['P=%.2f MeV/c' % pts[i,3] for i in range(len(pts))])
traces.append(go.Scatter3d(x=pts[:,0],y=pts[:,1],z=pts[:,2],
                           name='true edep',
                           marker=dict(size=2, color=pts[:,3], opacity=1.0),
                           line=dict(color='darkred',width=2),
                           hovertext = ['Edep=%.2f MeV' % pts[i,4] for i in range(len(pts))]
                          ))

# Shift the coordinates to show just a short track region
layout.scene.xaxis.range=(220,448)
layout.scene.yaxis.range=(-190,30)
layout.scene.zaxis.range=(650,650+220)


fig = go.Figure(data=traces)#,layout=layout)
fig.show()

In [ ]:
# look at groupings
